# Advanced Website Scraper - Usage Examples

This notebook demonstrates how to use the `AdvancedWebsite` class for web scraping.

## Features

- **Handles JavaScript-rendered pages** (React, Vue, etc.) using Playwright
- **Automatic fallback** to BeautifulSoup for static sites (faster)
- **Better error handling** with retry logic
- **Enhanced text extraction** with improved cleaning
- **Metadata extraction** (description, keywords, Open Graph tags)
- **Improved link extraction** with validation and normalization
- **Built-in summarization** using Ollama via OpenAI-compatible API

## Important Note

If you update `advanced_website_scraper.py` and the changes don't appear, **re-run Cell 2** (the import cell) to reload the module. The import cell includes automatic reloading, but you may need to re-run it after making changes to the Python file.


## Installation

If you want to use Playwright for JavaScript-rendered pages, you'll need to install it:

```bash
pip install playwright
playwright install
```

Note: Playwright is optional. The scraper will fall back to requests + BeautifulSoup if Playwright is not available.


In [ ]:
# Import the AdvancedWebsite class
# Note: This assumes advanced_website_scraper.py is in the same directory
# If running from a different location, you may need to adjust the import path
import sys
import os
import importlib

# Add current directory to path if needed
current_dir = os.path.dirname(os.path.abspath(''))
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Import the module
import advanced_website_scraper

# Reload the module to pick up any changes (useful during development)
importlib.reload(advanced_website_scraper)

# Import the class
from advanced_website_scraper import AdvancedWebsite

print("✓ AdvancedWebsite class imported successfully")
print(f"✓ Available methods: {[m for m in dir(AdvancedWebsite) if not m.startswith('_')]}")


## Configuration

Set your website URL and model name here. Change these variables to test different websites and models.


In [ ]:
# Configuration variables - Change these to test different websites and models
WEBSITE_URL = "https://streamlit.io"  # Change this to any website URL
MODEL_NAME = "llama3.1"  # Change this to your preferred Ollama model (e.g., "llama3.2", "llama3.1", etc.)
USE_JS = True  # Set to True to force JavaScript rendering (requires Playwright)

print(f"Configuration:")
print(f"  Website URL: {WEBSITE_URL}")
print(f"  Model Name: {MODEL_NAME}")
print(f"  Use JavaScript: {USE_JS}")


## Example 1: Scraping a Static Website (Fast)

For static websites, the scraper will use requests + BeautifulSoup, which is faster and lighter.

**Note:** This example uses the `WEBSITE_URL` variable defined in the configuration cell above.


In [ ]:
# Scrape a static website using the configured URL
website = AdvancedWebsite(WEBSITE_URL, use_js=USE_JS)

print(f"Fetch method: {website.get_fetch_method()}")
print(f"\nTitle: {website.title}")
print(f"\nContent preview (first 500 chars):\n{website.text[:500]}")
print(f"\nNumber of links found: {len(website.get_links())}")


## Example 2: Scraping a JavaScript-Rendered Website

For websites that use JavaScript to render content (like React, Vue, etc.), use Playwright mode.


In [ ]:
# Scrape a JavaScript-rendered website (like OpenAI)
# Note: This requires Playwright to be installed
try:
    website = AdvancedWebsite("https://openai.com", use_js=True, timeout=60)
    
    print(f"Fetch method: {website.get_fetch_method()}")
    print(f"\nTitle: {website.title}")
    print(f"\nContent preview (first 500 chars):\n{website.text[:500]}")
    print(f"\nNumber of links found: {len(website.get_links())}")
except Exception as e:
    print(f"Error: {e}")
    print("Make sure Playwright is installed: pip install playwright && playwright install")


## Example 3: Automatic Mode (Smart Fallback)

The scraper can automatically detect the best method. It tries requests first (faster), and falls back to Playwright if needed.


In [ ]:
# Let the scraper decide the best method using the configured URL
website = AdvancedWebsite(WEBSITE_URL)

print(f"Fetch method used: {website.get_fetch_method()}")
print(f"\nTitle: {website.title}")
print(f"\nContent preview (first 500 chars):\n{website.text[:500]}")


## Example 4: Getting Formatted Contents

Use the `get_contents()` method to get a formatted string with title and content.


In [ ]:
website = AdvancedWebsite(WEBSITE_URL)

# Get full contents
contents = website.get_contents()
print(contents[:1000])  # Print first 1000 characters

# Get contents with length limit
limited_contents = website.get_contents(max_length=500)
print("\n" + "="*50)
print("Limited to 500 characters:")
print(limited_contents)


## Example 5: Extracting Links

Get all links found on the page, with automatic validation and normalization.


In [ ]:
website = AdvancedWebsite("https://techcrunch.com/")

links = website.get_links()
print(f"Found {len(links)} links:")
for i, link in enumerate(links[:10], 1):  # Show first 10 links
    print(f"{i}. {link}")


## Example 6: Extracting Metadata

Extract metadata like description, keywords, Open Graph tags, and Twitter Card tags.


In [ ]:
website = AdvancedWebsite("https://openai.com")

metadata = website.get_metadata()
print("Extracted Metadata:")
print("=" * 50)
for key, value in metadata.items():
    print(f"{key}: {value}")


## Example 7: Waiting for Dynamic Content

If a page loads content dynamically, you can wait for a specific selector before extracting content.


In [ ]:
# Example: Wait for a specific element to load
# This is useful for pages that load content via JavaScript
# website = AdvancedWebsite(
#     WEBSITE_URL,
#     use_js=True,
#     wait_for_selector="main-content"  # Wait for element with this ID or class
# )


In [ ]:
# Compare the two approaches
from bs4 import BeautifulSoup
import requests

# Basic approach (from the original scraper)
def basic_scrape(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return title, text

# Advanced approach
url = "https://openai.com"
print("Basic scraper:")
basic_title, basic_text = basic_scrape(url)
print(f"Title: {basic_title}")
print(f"Text length: {len(basic_text)}")

print("\n" + "="*50)
print("Advanced scraper:")
advanced = AdvancedWebsite(url, use_js=False)
print(f"Title: {advanced.title}")
print(f"Text length: {len(advanced.text)}")
print(f"Links found: {len(advanced.get_links())}")
print(f"Metadata keys: {list(advanced.get_metadata().keys())}")


## Website Summarization with Ollama

The `AdvancedWebsite` class includes built-in summarization using Ollama via the OpenAI-compatible API. This allows you to automatically summarize any scraped website using local LLM models.


### Prerequisites for Summarization

Before using summarization, make sure:
1. Ollama is installed and running: `ollama serve`
2. A model is available: `ollama pull llama3.1` (or llama3.2, etc.)
3. OpenAI library is installed: `pip install openai`


## Example 8: Basic Website Summarization

Scrape a website and summarize it using Ollama.


In [ ]:
# Scrape and summarize a website
# Note: Some sites like openai.com have bot protection (403 errors)
# Use example.com or other sites for testing

try:
    # Use a simple site for testing (openai.com has bot protection)
    website = AdvancedWebsite("https://example.com", use_js=False)
    
    print("Website Title:", website.title)
    print("\n" + "="*50)
    print("Original Content (first 300 chars):")
    print(website.text[:300])
    print("\n" + "="*50)
    print("Summary using Ollama:")
    print("-" * 50)
    
    # Check if summarize method exists
    if hasattr(website, 'summarize'):
        # Summarize using default settings
        summary = website.summarize(model="llama3.1", temperature=0)
        print(summary)
    else:
        print("ERROR: summarize method not found!")
        print("\nSOLUTION: Please re-run the import cell (Cell 2) above to reload the module.")
        print("Or restart the kernel: Kernel -> Restart Kernel")
        print(f"\nAvailable methods: {[m for m in dir(website) if not m.startswith('_')]}")
    
except Exception as e:
    error_msg = str(e)
    print(f"Error: {error_msg}")
    
    if "'AdvancedWebsite' object has no attribute 'summarize'" in error_msg:
        print("\n" + "="*60)
        print("SOLUTION: The module needs to be reloaded!")
        print("="*60)
        print("1. Go back to Cell 2 (the import cell)")
        print("2. Re-run that cell (Shift+Enter)")
        print("3. Then come back and run this cell again")
        print("\nOR restart the kernel:")
        print("   Kernel -> Restart Kernel -> Restart")
    else:
        print("\nTroubleshooting:")
        print("1. Make sure Ollama is running: ollama serve")
        print("2. Make sure model is available: ollama pull llama3.1")
        print("3. Make sure OpenAI library is installed: pip install openai")


## Example 9: Advanced Summarization with Custom Settings

Use custom prompts, temperature, and other parameters for summarization.


In [ ]:
try:
    website = AdvancedWebsite(WEBSITE_URL, use_js=USE_JS)
    
    # Custom system prompt for more detailed summary
    custom_prompt = (
        "You are an expert content analyst. Provide a comprehensive summary "
        "that includes: 1) Main topic, 2) Key points, 3) Important details, "
        "4) Any actionable insights. Format your response clearly."
    )
    
    summary = website.summarize_with_ollama(
        model=MODEL_NAME,
        temperature=0.3,  # Slightly more creative
        max_tokens=500,   # Limit response length
        system_prompt=custom_prompt
    )
    
    print("Detailed Summary:")
    print("=" * 50)
    print(summary)
    
except Exception as e:
    print(f"Error: {e}")


## Example 10: Summarizing JavaScript-Rendered Websites

Combine advanced scraping with summarization for JavaScript-heavy sites.


In [ ]:
# Scrape a JavaScript-rendered site and summarize it
try:
    # Note: This requires Playwright to be installed
    website = AdvancedWebsite(WEBSITE_URL, use_js=USE_JS)
    
    print(f"Scraped using: {website.get_fetch_method()}")
    print(f"Title: {website.title}")
    print(f"Content length: {len(website.text)} characters")
    print("\n" + "="*50)
    print("Summary:")
    print("-" * 50)
    
    summary = website.summarize(model="llama3.1", temperature=0)
    print(summary)
    
except Exception as e:
    print(f"Error: {e}")
    print("\nIf you see Playwright errors, you can:")
    print("1. Install Playwright: pip install playwright && playwright install")
    print("2. Or use use_js=False for static sites")


## Example 11: Complete Workflow - Scrape, Extract, and Summarize

A complete example showing the full workflow from scraping to summarization.


In [ ]:
def scrape_and_summarize(url, model, use_js=False):
    """
    Complete workflow: Scrape a website and summarize it.
    
    Args:
        url: URL to scrape
        model: Ollama model to use for summarization
        use_js: Whether to use JavaScript rendering
    
    Returns:
        Dictionary with scraped data and summary
    """
    try:
        # Step 1: Scrape the website
        print(f"Scraping {url}...")
        website = AdvancedWebsite(url, use_js=use_js)
        print(f"✓ Scraped using: {website.get_fetch_method()}")
        
        # Step 2: Extract information
        print(f"✓ Title: {website.title}")
        print(f"✓ Content length: {len(website.text)} characters")
        print(f"✓ Links found: {len(website.get_links())}")
        print(f"✓ Metadata keys: {list(website.get_metadata().keys())}")
        
        # Step 3: Summarize
        print(f"\nGenerating summary using {model}...")
        summary = website.summarize(model=model, temperature=0)
        print("✓ Summary generated")
        
        return {
            "url": url,
            "title": website.title,
            "content_length": len(website.text),
            "links_count": len(website.get_links()),
            "metadata": website.get_metadata(),
            "summary": summary,
            "fetch_method": website.get_fetch_method()
        }
        
    except Exception as e:
        print(f"✗ Error: {e}")
        return None

# Example usage with configured variables
result = scrape_and_summarize(WEBSITE_URL, model=MODEL_NAME, use_js=USE_JS)

if result:
    print("\n" + "="*50)
    print("SUMMARY:")
    print("="*50)
    print(result["summary"])


## Summary

The `AdvancedWebsite` class now provides a complete solution for web scraping and summarization:

1. **Advanced Scraping**: Handles both static and JavaScript-rendered pages
2. **Smart Fallback**: Automatically chooses the best method
3. **Enhanced Extraction**: Better text cleaning, link validation, metadata extraction
4. **Built-in Summarization**: Summarize any scraped website using Ollama
5. **Flexible Configuration**: Customize all aspects of scraping and summarization

### Key Methods:
- `AdvancedWebsite(url)` - Scrape a website
- `website.summarize()` - Quick summarization with defaults
- `website.summarize_with_ollama()` - Advanced summarization with custom settings
- `website.get_contents()` - Get formatted content
- `website.get_links()` - Get all links
- `website.get_metadata()` - Get page metadata
